In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from collections import Counter
import json

In [38]:
# First declare which split we're working on

SPLIT = "validation"


In [39]:
# Get file names and sort them to get them in the right order
filepath = f"../data/many_emotions/finnish_translations/{SPLIT}"
file_names = [file for file in os.listdir(filepath) if file.endswith(".txt")]
file_names.sort(key=lambda file: file[-6:-4])
file_names

['many_emotions_validation_00.txt',
 'many_emotions_validation_01.txt',
 'many_emotions_validation_02.txt',
 'many_emotions_validation_03.txt',
 'many_emotions_validation_04.txt']

In [40]:
data = []
for name in file_names:
    with open(os.path.join(filepath, name), "r") as f:
        file = f.readlines()
        for item in file:
            data.append(item)

In [41]:
headers = []
content = []
for idx, line in enumerate(data):
    if idx % 2 == 0:
        headers.append(line)
    else:
        content.append(line)

In [42]:
# Remove trailing newline characters
content = [text.strip("\n") for text in content]
content[0:10]

['Sinulla näyttää olevan hyvin rento asenne.',
 'tunnen itseni hyvin epävarmaksi dps-tehtävässä -',
 'Tiedän, että ne voivat helposti pahentua paljon pahemmiksi kuin ne ovat nyt, mutta tunnen itseni todella kurjaksi.',
 'minusta tuntuu melkein, että se on eräänlainen homofobian muoto, tämä oletus, että pystymme huolehtimaan itsestämme jopa lapsina teini-ikäisinä, joten älkää huolehtiko meidän tukemisestamme.',
 'tunnen kiitollisuutta aamuisin, kun katsomme talomme ylhäältä alas.',
 'Näin ja tunsin rintojen hellästi paisuvan kaapin alla...',
 'en tunne kipua se on kaukana tylsästä',
 'Yritän edelleen, koska minusta tuntuu, että minun on tehtävä onnistunut pottaan meno Shanen ulottuville, jotta olisin kunnon isä.',
 'minulla on ja keskityn kaikkiin haluihini ja siihen, mitä tunnen tarvitsevani, mutta kun tämä vuodenaika koittaa, minua muistutetaan jälleen siitä, kuinka siunattu olen ja kuinka minulla on niin paljon enemmän kuin tarvitsen.',
 'haluan pukeutua johonkin glamouriseen']

In [43]:
# Load in the original dataset with the labels
df = pd.read_json(f"../data/many_emotions/many_emotions_en/many_emotions_{SPLIT}.jsonl", lines=True)
df

,id,text,label,dataset,license,language
0,daily_dialog_0_7994_4_en,You seem to have a very relaxed outlook .,6,daily_dialog,CC BY-NC-SA 4.0,0
1,emotion_403281_en,i feel very shaky on dps duty,1,emotion,educational/research,0
2,emotion_299785_en,i know they can easily get a lot worse than th...,4,emotion,educational/research,0
3,emotion_280064_en,i almost feel like its a form of homophobia th...,3,emotion,educational/research,0
4,emotion_323682_en,i feel thankful in the mornings when we look d...,2,emotion,educational/research,0
...,...,...,...,...,...,...
27409,emotion_118957_en,i grabbed the pen feeling regretful and wrote ...,4,emotion,educational/research,0
27410,emotion_231277_en,i kinda wish i was feeling more artistic lately,2,emotion,educational/research,0
27411,emotion_279477_en,i feel ecstatic dont worry we can make love au...,2,emotion,educational/research,0
27412,emotion_171877_en,i use this after the purifying toner and my sk...,5,emotion,educational/research,0


In [44]:
# Replace English text with Finnish text
df["text"] = content
df

,id,text,label,dataset,license,language
0,daily_dialog_0_7994_4_en,Sinulla näyttää olevan hyvin rento asenne.,6,daily_dialog,CC BY-NC-SA 4.0,0
1,emotion_403281_en,tunnen itseni hyvin epävarmaksi dps-tehtävässä -,1,emotion,educational/research,0
2,emotion_299785_en,"Tiedän, että ne voivat helposti pahentua paljo...",4,emotion,educational/research,0
3,emotion_280064_en,"minusta tuntuu melkein, että se on eräänlainen...",3,emotion,educational/research,0
4,emotion_323682_en,"tunnen kiitollisuutta aamuisin, kun katsomme t...",2,emotion,educational/research,0
...,...,...,...,...,...,...
27409,emotion_118957_en,"tartuin kynään katuen ja kirjoitin ""anteeksi"".",4,emotion,educational/research,0
27410,emotion_231277_en,"Toivon, että olisin viime aikoina ollut taitee...",2,emotion,educational/research,0
27411,emotion_279477_en,tunnen itseni ekstaattiseksi älä huoli voimme ...,2,emotion,educational/research,0
27412,emotion_171877_en,"käytän tätä puhdistavan kasvoveden jälkeen, ja...",5,emotion,educational/research,0


In [45]:
# Update sentence ids to reflect the language change to avoid confusion later on

ids = df["id"].to_list()
for x,i in enumerate(ids):
    new_id = i[0:-2] + "fi" # replace the last 2 character with "fi"
    ids[x] = new_id

print(ids[0:20])

['daily_dialog_0_7994_4_fi', 'emotion_403281_fi', 'emotion_299785_fi', 'emotion_280064_fi', 'emotion_323682_fi', 'emotion_280980_fi', 'emotion_368094_fi', 'emotion_154259_fi', 'emotion_416338_fi', 'emotion_76056_fi', 'go_emotions_1_20650_fi', 'emotion_255632_fi', 'emotion_91048_fi', 'emotion_189706_fi', 'emotion_263016_fi', 'emotion_361699_fi', 'emotion_264532_fi', 'emotion_394608_fi', 'emotion_65713_fi', 'go_emotions_0_7405_fi']


In [46]:
# Place new ids in dataframe
# Drop redundant language column

df["id"] = ids
df.drop("language", axis = 1, inplace = True)
df

,id,text,label,dataset,license
0,daily_dialog_0_7994_4_fi,Sinulla näyttää olevan hyvin rento asenne.,6,daily_dialog,CC BY-NC-SA 4.0
1,emotion_403281_fi,tunnen itseni hyvin epävarmaksi dps-tehtävässä -,1,emotion,educational/research
2,emotion_299785_fi,"Tiedän, että ne voivat helposti pahentua paljo...",4,emotion,educational/research
3,emotion_280064_fi,"minusta tuntuu melkein, että se on eräänlainen...",3,emotion,educational/research
4,emotion_323682_fi,"tunnen kiitollisuutta aamuisin, kun katsomme t...",2,emotion,educational/research
...,...,...,...,...,...
27409,emotion_118957_fi,"tartuin kynään katuen ja kirjoitin ""anteeksi"".",4,emotion,educational/research
27410,emotion_231277_fi,"Toivon, että olisin viime aikoina ollut taitee...",2,emotion,educational/research
27411,emotion_279477_fi,tunnen itseni ekstaattiseksi älä huoli voimme ...,2,emotion,educational/research
27412,emotion_171877_fi,"käytän tätä puhdistavan kasvoveden jälkeen, ja...",5,emotion,educational/research


In [47]:
df.to_csv(f"../data/many_emotions/many_emotions_fi_{SPLIT}.tsv", index = False, sep = "\t")